In [1]:
'''
This set of scripts preprocesses .nd2 files to extract metadata then
split nd2 files into single channel multipage tiff files

all you need is name your images carefully during nikon image acquisition as follows in example below:

two channels image:

"unimportant"_ch0-Nuclei-405_"unimportant"_ch1-Ryr2-568_"unimportant".nd2
example:
2024-11-22_WT-Male1_ch0-Nuclei-405_ch1-Ryr2-568_cell11.nd2

three channels image:

"unimportant"_ch0-Nuclei-405_"unimportant"_ch1-RnaRyr2-568_"unimportant"_ch2-ProtRyr2-568_"unimportant".nd2
example:
2024-11-22_WT-Male1_ch0-Nuclei-405_ch1-Ryr2-488_ch2-ProtRyr2-568_cell11.nd2

four channels image:

"unimportant"_ch0-Nuclei-405_"unimportant"_ch1-RnaRyr2-488_"unimportant"_ch2-ProtRyr2-568_"unimportant"_ch3-RnaScn5a-647_"unimportant".nd2
example:
2024-11-22_WT-Male1_ch0-Nuclei-405_ch1-Ryr2-568_ch2-ProtRyr2-568_cell11.nd2

'''


'''
Import all these libraries first
'''

import os
import xml.etree.ElementTree as ET
from nd2 import ND2File
import re
from nd2reader import ND2Reader
import tifffile
import concurrent.futures



In [ ]:
'''this script replaces specific words in the name of all your files
within specified directory
do if you want to rename any parts of your files'''
# Define the input directory
input_directory = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3'  # Update your directory path

# Function to replace "Calm2" with "Calm3" in filenames
def rename_files(directory):
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
#             if filename.endswith(".tif") and "Calm2" in filename:
            if filename.endswith(".tif"):                
                new_filename = filename.replace("old_word_in_file_name", "new_word_in_file_name")
                old_file_path = os.path.join(dirpath, filename)
                new_file_path = os.path.join(dirpath, new_filename)
                os.rename(old_file_path, new_file_path)
                # print(f'Renamed "{old_file_path}" to "{new_file_path}"')

# Rename the files
rename_files(input_directory)


In [ ]:

'''
this code moves all your .nd2 files into the same directory and run this code. 
this code will create folders with the same name as your file names
and will move these files into the corresponding folders
'''

def get_nd2_paths_in_directory(directory):
    nd2_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".nd2")]
    return nd2_paths

def create_folder_and_move_nd2(nd2_path, output_dir):
    filename = os.path.basename(nd2_path)
    folder_name, _ = os.path.splitext(filename)
    folder_path = os.path.join(output_dir, folder_name)

    os.makedirs(folder_path, exist_ok=True)
    shutil.move(nd2_path, os.path.join(folder_path, filename))

def process_nd2_files_in_directory(directory, output_dir):
    nd2_paths = get_nd2_paths_in_directory(directory)
    for nd2_path in nd2_paths:
        create_folder_and_move_nd2(nd2_path, output_dir)

if __name__ == "__main__":

    input_directory = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3' # replace with path to your directory where your images for segmentation are located 
    output_directory = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3' # replace with path to your directory where your images for segmentation are located
    
    process_nd2_files_in_directory(input_directory, output_directory)



In [2]:
# creates a list of paths (path_list) with working FOLDERS


def get_nd2_paths_in_directory():

#     input_dir = fr'\\?\D:\Official_Manuscript_Data\figure4\Calms\fig4_Same_Data_Another_Order'

#    input_dir = fr'\\?\D:\Projects\Grants\AHA_Postdoctoral_RK01_2023'
    input_dir = fr'D:\Projects\ML_Segmentation_Training\Calms\Nuclei\raw_images_2D'
#     input_dir = fr'\\?\D:\Projects\Inducible_NOS\Calm1_Nos_project\WT_B57BL6_control___RNA_probes_were_old\570'

    path_list = []

    # Loop through all directories and subdirectories inside the input directory
    for root, dirs, files in os.walk(input_dir):
        # Loop through all files in the current directory
        for file in files:
            # Check if the file has an _info.txt extension and starts with the folder name
            if ".nd2" in file and file.startswith(os.path.basename(root)):
                # Read the file content and convert it to lowercase
                
                path_list.append(root)

    # Return the list of paths
    return path_list



In [3]:
# runs function get_path and saves path list as .txt file

import os

# runs function get_path and saves path list as .txt file
input_dir = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3' # replace with path to your directory where your images for segmentation are located

path_folders_list = get_nd2_paths_in_directory()

txt_file_path = os.path.join(input_dir, 'path_list.txt')

with open(txt_file_path, 'w') as file:
    for item in path_folders_list:
        file.write(item + '\n')

len(path_folders_list)


12

In [4]:
path_folders_list # to check the list of your folders

['\\\\?\\D:\\Projects\\LocProt\\ISH_IF\\IF_ISH_Fig3\\2023-11-27_C57BL6_M1_Gja1-488_Calm2-568_Cx43-647_cell11_deconv',
 '\\\\?\\D:\\Projects\\LocProt\\ISH_IF\\IF_ISH_Fig3\\2023-11-27_C57BL6_M1_Gja1-488_Calm2-568_Cx43-647_cell12_deconv',
 '\\\\?\\D:\\Projects\\LocProt\\ISH_IF\\IF_ISH_Fig3\\2023-11-27_C57BL6_M1_Gja1-488_Calm2-568_Cx43-647_cell13_deconv',
 '\\\\?\\D:\\Projects\\LocProt\\ISH_IF\\IF_ISH_Fig3\\2023-11-27_C57BL6_M1_MRyr2-488_Calm2-568_PRyR2-647_cell1_deconv',
 '\\\\?\\D:\\Projects\\LocProt\\ISH_IF\\IF_ISH_Fig3\\2023-11-27_C57BL6_M1_MRyr2-488_Calm2-568_PRyR2-647_cell2_deconv',
 '\\\\?\\D:\\Projects\\LocProt\\ISH_IF\\IF_ISH_Fig3\\2023-11-27_C57BL6_M1_MRyr2-488_Calm2-568_PRyR2-647_cell3_deconv',
 '\\\\?\\D:\\Projects\\LocProt\\ISH_IF\\IF_ISH_Fig3\\2023-11-27_C57BL6_M2_Gja1-488_Calm2-568_Cx43-647_cell1_deconv',
 '\\\\?\\D:\\Projects\\LocProt\\ISH_IF\\IF_ISH_Fig3\\2023-11-27_C57BL6_M2_Gja1-488_Calm2-568_Cx43-647_cell2_deconv',
 '\\\\?\\D:\\Projects\\LocProt\\ISH_IF\\IF_ISH_Fig3\\20

In [1]:
# use this function to remove any .tif or .xml or .txt files in path_list
'''
!!!Use this script very carefully, because it will remove all your files with extension you specified 
from the directory you specified under input_dir!!!

uncomment this script if you want to use it
'''

# import os
# input_dir = fr'\\?\D:\Projects\Calm_ISO\Vlad_Dataset_Organized\Again' # path from where to remove all files with specific extension

# def remove_tif_files(directories):
#     for directory in directories:
#         for file in os.listdir(directory):
#             if file.endswith(".txt"):
#                 file_path = os.path.join(directory, file)
#                 os.remove(file_path)

# if __name__ == "__main__":
#     directories = path_folders_list
#     remove_tif_files(directories)


'\n!!!Use this script very carefully, because it will remove all your files with extension you specified \nfrom the directory you specified under input_dir!!!\n\nuncomment this script if you want to use it\n'

In [5]:
# use this script to extract Metadata from nd2 and save file as xml file using path_folders_list


import os
import xml.etree.ElementTree as ET
from nd2 import ND2File

def extract_Metadata_from_nd2(path_folders_list):
    for path in path_folders_list:
        # Find the .nd2 file with the same name as the folder
        nd2_filename = os.path.basename(path) + ".nd2"
        nd2_file_path = os.path.join(path, nd2_filename)

        with ND2File(nd2_file_path) as nd2_file:
            metadata = nd2_file.metadata
            root = ET.Element("Metadata")
            for key, value in metadata.__dict__.items():
                elem = ET.SubElement(root, key)
                elem.text = str(value)

            output_path = os.path.join(path, os.path.basename(path) + '.xml')
            tree = ET.ElementTree(root)
            tree.write(output_path, encoding='utf-8', xml_declaration=True)

# Example usage:
extract_Metadata_from_nd2(path_folders_list)


In [11]:
# # use to replace word in _info.txt file

# import os

# # the root directory where your .txt files are located
# # root_directory = fr'\\?\D:\Projects\For_Jon'
# root_directory = fr'D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3'

# # the word you want to replace
# old_word = "Calm2"

# # the new word you want to replace with
# new_word = "Calm3"

# # loop through all directories and subdirectories in the root directory
# for dirpath, dirnames, filenames in os.walk(root_directory):
#     # loop through all files in the current directory
#     for filename in filenames:
#         if filename.endswith(".txt"):
#             # open the file and read its contents
#             with open(os.path.join(dirpath, filename), "r") as file:
#                 file_contents = file.read()

#             # replace the old word with the new word in the file contents
#             file_contents = file_contents.replace(old_word, new_word)

#             # write the modified contents back to the file
#             with open(os.path.join(dirpath, filename), "w") as file:
#                 file.write(file_contents)


In [6]:
# use this script to extract data from xml file and create _info.txt file

import os
import re
import xml.etree.ElementTree as ET

def extract_Metadata_from_nd2(path_list):
    for folder_path in path_list:
        # Find the .nd2 file with the same name as the folder
        folder_name = os.path.basename(folder_path)
        nd2_filename = folder_name + ".nd2"
        
        # Read the XML file
        xml_file_path = os.path.join(folder_path, folder_name + ".xml")
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        
        # Extract axes calibration values from the XML file
        axes_calibration_str = root.find("channels").text
        axes_calibration = re.search(r"axesCalibration=\[(.+?), (.+?), (.+?)\]", axes_calibration_str)
        xy_cal = round(float(axes_calibration.group(1)), 3)
        z_cal = round(float(axes_calibration.group(3)), 3)

        channel_n1 = ""
        channel_n2 = ""
        channel_n3 = ""
        ExperimentalGroup = ""
        
        if "405" in nd2_filename:
            channel_n0 = re.findall(r"([0-9A-Za-z]+)[-|_]405", nd2_filename)[0]
        if "488" in nd2_filename:
            channel_n1 = re.findall(r"([0-9A-Za-z]+)[-|_]488", nd2_filename)[0]
        if "568" in nd2_filename:
            channel_n2 = re.findall(r"([0-9A-Za-z]+)[-|_]568", nd2_filename)[0]
        if "647" in nd2_filename:
            channel_n3 = re.findall(r"([0-9A-Za-z]+)[-|_]647", nd2_filename)[0]
            

        ExperimentalGroup = "WT" # replace with anything you want
            
        # Write the information to a txt file
        txt_output_path = os.path.join(folder_path, folder_name + '_info.txt')

        #for 4 channels image. Replace "405", "488", "568" and "647" with your own labels
        if "405" in nd2_filename and "488" in nd2_filename and "568" in nd2_filename and "647" in nd2_filename:
            with open(txt_output_path, 'w') as txt_file:
                txt_file.write(f"ChannelNumbers:0,1,2,3:ChannelNames:Nuclei,{channel_n1},{channel_n2},{channel_n3}:XYZCalibration(um/vox):{xy_cal},{xy_cal},{z_cal}:ExperimentalGroup:{ExperimentalGroup}:CellID:\n")

        #for 3 channels image. Replace "405", "488" and "568" with your own labels
    #     if "405" in nd2_filename and "488" in nd2_filename and "568" in nd2_filename: 
    #         with open(txt_output_path, 'w') as txt_file:
    # #             txt_file.write(f"ChannelNumbers:0,1,2:ChannelNames:nucleus,{channel_n1},{channel_n2}:XYZCalibration(um/vox):{xy_cal},{xy_cal},{z_cal}:ExperimentalGroup:{ExperimentalGroup}:CellID:\n")
                
        
# Example usage:
extract_Metadata_from_nd2(path_folders_list)


In [7]:
# IMPORTANT SCRIPT!!! this code converts nd2 file into tif and keeps metadata with PARALLELIZATION

def process_folder(folder_path):
    # Load the nd2 file
    folder_name = os.path.basename(folder_path)
    nd2_filename = folder_name + ".nd2"
    nd2_file_path = os.path.join(folder_path, nd2_filename)
    images = ND2Reader(nd2_file_path)

    # Read the XML file
    xml_file_path = os.path.join(folder_path, folder_name + ".xml")
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Extract number of channels from the XML file
    channels_str = root.find("contents").text
    num_channels = int(re.search(r"channelCount=(\d+)", channels_str).group(1))
#     print(f"{num_channels}+")
    # Iterate through channels and save them as separate tif files
    for channel in range(num_channels):

        # Select the current channel
        images.default_coords['c'] = channel

        # Create a tif file name based on the channel number
        tif_file_name = os.path.basename(folder_path) + f"_ch0{channel}.tif"
        tif_file_path = os.path.join(folder_path, tif_file_name)

        # Save the current channel as a tif file
        tifffile.imwrite(tif_file_path, images)

    print(f"Tif files saved successfully for folder: {folder_name}")


def split_nd2_file_into_channels_parallel(path_list, max_workers=32):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(process_folder, path_list)



In [8]:
len(path_folders_list) # to check the number of folders to process


12

In [9]:
# Example usage:
split_nd2_file_into_channels_parallel(path_folders_list)


Tif files saved successfully for folder: 2023-11-27_C57BL6_M2_Gja1-488_Calm2-568_Cx43-647_cell1_deconv
Tif files saved successfully for folder: 2023-11-27_C57BL6_M2_MRyr2-488_Calm2-568_PRyR2-647_cell1_deconv
Tif files saved successfully for folder: 2023-11-27_C57BL6_M1_MRyr2-488_Calm2-568_PRyR2-647_cell3_deconv
Tif files saved successfully for folder: 2023-11-27_C57BL6_M1_MRyr2-488_Calm2-568_PRyR2-647_cell1_deconv
Tif files saved successfully for folder: 2023-11-27_C57BL6_M1_MRyr2-488_Calm2-568_PRyR2-647_cell2_deconv
Tif files saved successfully for folder: 2023-11-27_C57BL6_M2_MRyr2-488_Calm2-568_PRyR2-647_cell2_deconv
Tif files saved successfully for folder: 2023-11-27_C57BL6_M2_Gja1-488_Calm2-568_Cx43-647_cell2_deconv
Tif files saved successfully for folder: 2023-11-27_C57BL6_M1_Gja1-488_Calm2-568_Cx43-647_cell13_deconv
Tif files saved successfully for folder: 2023-11-27_C57BL6_M2_Gja1-488_Calm2-568_Cx43-647_cell3_deconv
Tif files saved successfully for folder: 2023-11-27_C57BL6_M1_